In [1]:
%%time

! pip install datasketch

Defaulting to user installation because normal site-packages is not writeable
CPU times: total: 0 ns
Wall time: 1.19 s



[notice] A new release of pip is available: 25.0.1 -> 25.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import numpy as np
import pandas as pd
import time
import re
import string
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.model_selection import train_test_split
from datasketch import MinHash
from datasketch import MinHashLSHForest

In [3]:
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Main\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Main\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Main\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [4]:
#Import the CSV file
df = pd.read_csv("https://raw.githubusercontent.com/LaertXh/LLM-Project/refs/heads/main/Data/rotten_tomatoes_movies.csv")

In [5]:
df.shape

(17712, 22)

In [6]:
df.head()

,rotten_tomatoes_link,movie_title,movie_info,critics_consensus,content_rating,genres,directors,authors,actors,original_release_date,...,production_company,tomatometer_status,tomatometer_rating,tomatometer_count,audience_status,audience_rating,audience_count,tomatometer_top_critics_count,tomatometer_fresh_critics_count,tomatometer_rotten_critics_count
0,m/0814255,Percy Jackson & the Olympians: The Lightning T...,"Always trouble-prone, the life of teenager Per...",Though it may seem like just another Harry Pot...,PG,"Action & Adventure, Comedy, Drama, Science Fic...",Chris Columbus,"Craig Titley, Chris Columbus, Rick Riordan","Logan Lerman, Brandon T. Jackson, Alexandra Da...",2010-02-12,...,20th Century Fox,Rotten,49.0,149.0,Spilled,53.0,254421.0,43,73,76
1,m/0878835,Please Give,Kate (Catherine Keener) and her husband Alex (...,Nicole Holofcener's newest might seem slight i...,R,Comedy,Nicole Holofcener,Nicole Holofcener,"Catherine Keener, Amanda Peet, Oliver Platt, R...",2010-04-30,...,Sony Pictures Classics,Certified-Fresh,87.0,142.0,Upright,64.0,11574.0,44,123,19
2,m/10,10,"A successful, middle-aged Hollywood songwriter...",Blake Edwards' bawdy comedy may not score a pe...,R,"Comedy, Romance",Blake Edwards,Blake Edwards,"Dudley Moore, Bo Derek, Julie Andrews, Robert ...",1979-10-05,...,Waner Bros.,Fresh,67.0,24.0,Spilled,53.0,14684.0,2,16,8
3,m/1000013-12_angry_men,12 Angry Men (Twelve Angry Men),Following the closing arguments in a murder tr...,Sidney Lumet's feature debut is a superbly wri...,NR,"Classics, Drama",Sidney Lumet,Reginald Rose,"Martin Balsam, John Fiedler, Lee J. Cobb, E.G....",1957-04-13,...,Criterion Collection,Certified-Fresh,100.0,54.0,Upright,97.0,105386.0,6,54,0
4,m/1000079-20000_leagues_under_the_sea,"20,000 Leagues Under The Sea","In 1866, Professor Pierre M. Aronnax (Paul Luk...","One of Disney's finest live-action adventures,...",G,"Action & Adventure, Drama, Kids & Family",Richard Fleischer,Earl Felton,"James Mason, Kirk Douglas, Paul Lukas, Peter L...",1954-01-01,...,Disney,Fresh,89.0,27.0,Upright,74.0,68918.0,5,24,3


In [7]:
#DATASET CLEANING

In [8]:
#Drop unneeded columns
df.drop(['rotten_tomatoes_link', 'original_release_date', 'streaming_release_date', 'runtime', 'authors','production_company', 'tomatometer_status', 'tomatometer_rating', 'tomatometer_count', 'audience_status', 'audience_rating', 'audience_count', 'tomatometer_top_critics_count', 'tomatometer_fresh_critics_count', 'tomatometer_rotten_critics_count'], axis=1, inplace = True)


In [9]:
df.head()

,movie_title,movie_info,critics_consensus,content_rating,genres,directors,actors
0,Percy Jackson & the Olympians: The Lightning T...,"Always trouble-prone, the life of teenager Per...",Though it may seem like just another Harry Pot...,PG,"Action & Adventure, Comedy, Drama, Science Fic...",Chris Columbus,"Logan Lerman, Brandon T. Jackson, Alexandra Da..."
1,Please Give,Kate (Catherine Keener) and her husband Alex (...,Nicole Holofcener's newest might seem slight i...,R,Comedy,Nicole Holofcener,"Catherine Keener, Amanda Peet, Oliver Platt, R..."
2,10,"A successful, middle-aged Hollywood songwriter...",Blake Edwards' bawdy comedy may not score a pe...,R,"Comedy, Romance",Blake Edwards,"Dudley Moore, Bo Derek, Julie Andrews, Robert ..."
3,12 Angry Men (Twelve Angry Men),Following the closing arguments in a murder tr...,Sidney Lumet's feature debut is a superbly wri...,NR,"Classics, Drama",Sidney Lumet,"Martin Balsam, John Fiedler, Lee J. Cobb, E.G...."
4,"20,000 Leagues Under The Sea","In 1866, Professor Pierre M. Aronnax (Paul Luk...","One of Disney's finest live-action adventures,...",G,"Action & Adventure, Drama, Kids & Family",Richard Fleischer,"James Mason, Kirk Douglas, Paul Lukas, Peter L..."


In [10]:
#Limit the number of directors to the first listed director
df['directors'] = df['directors'].str.split(',').str[0]

In [11]:
df['directors'].head()

0       Chris Columbus
1    Nicole Holofcener
2        Blake Edwards
3         Sidney Lumet
4    Richard Fleischer
Name: directors, dtype: object

In [12]:
#Limit the number of actors to the first four listed actors
df['actors'] = df['actors'].str.split(',').str[:4].str.join(',')

In [13]:
df['actors'].head()

0    Logan Lerman, Brandon T. Jackson, Alexandra Da...
1    Catherine Keener, Amanda Peet, Oliver Platt, R...
2    Dudley Moore, Bo Derek, Julie Andrews, Robert ...
3    Martin Balsam, John Fiedler, Lee J. Cobb, E.G....
4    James Mason, Kirk Douglas, Paul Lukas, Peter L...
Name: actors, dtype: object

In [14]:
df.head()

,movie_title,movie_info,critics_consensus,content_rating,genres,directors,actors
0,Percy Jackson & the Olympians: The Lightning T...,"Always trouble-prone, the life of teenager Per...",Though it may seem like just another Harry Pot...,PG,"Action & Adventure, Comedy, Drama, Science Fic...",Chris Columbus,"Logan Lerman, Brandon T. Jackson, Alexandra Da..."
1,Please Give,Kate (Catherine Keener) and her husband Alex (...,Nicole Holofcener's newest might seem slight i...,R,Comedy,Nicole Holofcener,"Catherine Keener, Amanda Peet, Oliver Platt, R..."
2,10,"A successful, middle-aged Hollywood songwriter...",Blake Edwards' bawdy comedy may not score a pe...,R,"Comedy, Romance",Blake Edwards,"Dudley Moore, Bo Derek, Julie Andrews, Robert ..."
3,12 Angry Men (Twelve Angry Men),Following the closing arguments in a murder tr...,Sidney Lumet's feature debut is a superbly wri...,NR,"Classics, Drama",Sidney Lumet,"Martin Balsam, John Fiedler, Lee J. Cobb, E.G...."
4,"20,000 Leagues Under The Sea","In 1866, Professor Pierre M. Aronnax (Paul Luk...","One of Disney's finest live-action adventures,...",G,"Action & Adventure, Drama, Kids & Family",Richard Fleischer,"James Mason, Kirk Douglas, Paul Lukas, Peter L..."


In [15]:
#Check for duplicate rows
duplicate_rows = df[df.duplicated()]

In [16]:
print(duplicate_rows)

Empty DataFrame
Columns: [movie_title, movie_info, critics_consensus, content_rating, genres, directors, actors]
Index: []


In [17]:
#Identify null values
df.isnull().sum()

movie_title             0
movie_info            321
critics_consensus    8578
content_rating          0
genres                 19
directors             194
actors                352
dtype: int64

In [18]:
#Drop rows with null values
df_cleaned = df.dropna(subset=['movie_info', 'genres', 'directors', 'actors'])

In [19]:
#Drop the critics_consensus column
df_cleaned.drop(['critics_consensus'], axis=1, inplace = True)

C:\Users\Main\AppData\Local\Temp\ipykernel_22868\3888570728.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cleaned.drop(['critics_consensus'], axis=1, inplace = True)


In [20]:
df_cleaned.isnull().sum()

movie_title       0
movie_info        0
content_rating    0
genres            0
directors         0
actors            0
dtype: int64

In [21]:
df_cleaned.head()

,movie_title,movie_info,content_rating,genres,directors,actors
0,Percy Jackson & the Olympians: The Lightning T...,"Always trouble-prone, the life of teenager Per...",PG,"Action & Adventure, Comedy, Drama, Science Fic...",Chris Columbus,"Logan Lerman, Brandon T. Jackson, Alexandra Da..."
1,Please Give,Kate (Catherine Keener) and her husband Alex (...,R,Comedy,Nicole Holofcener,"Catherine Keener, Amanda Peet, Oliver Platt, R..."
2,10,"A successful, middle-aged Hollywood songwriter...",R,"Comedy, Romance",Blake Edwards,"Dudley Moore, Bo Derek, Julie Andrews, Robert ..."
3,12 Angry Men (Twelve Angry Men),Following the closing arguments in a murder tr...,NR,"Classics, Drama",Sidney Lumet,"Martin Balsam, John Fiedler, Lee J. Cobb, E.G...."
4,"20,000 Leagues Under The Sea","In 1866, Professor Pierre M. Aronnax (Paul Luk...",G,"Action & Adventure, Drama, Kids & Family",Richard Fleischer,"James Mason, Kirk Douglas, Paul Lukas, Peter L..."


In [22]:
#LSH CLEANING

In [23]:
#MOVIE_INFO CLEANING

In [24]:
pd.set_option('display.max_colwidth', None)

In [25]:
df_cleaned['movie_info'].head()

0                                       Always trouble-prone, the life of teenager Percy Jackson (Logan Lerman) gets a lot more complicated when he learns he's the son of the Greek god Poseidon. At a training ground for the children of deities, Percy learns to harness his divine powers and prepare for the adventure of a lifetime: he must prevent a feud among the Olympians from erupting into a devastating war on Earth, and rescue his mother from the clutches of Hades, god of the underworld.
1       Kate (Catherine Keener) and her husband Alex (Oliver Platt) are wealthy New Yorkers who prowl estate sales and make a tidy profit reselling items they bought cheaply. They buy the apartment next door and plan to remodel just as soon as its current occupant, a cranky old woman, dies. Kate is troubled by the way she and her husband earn a living, and tries to assuage her guilt by befriending her tenant and the woman's granddaughters, but her overtures lead to unexpected consequences.
2         

In [26]:
df_cleaned.shape

(16908, 6)

In [27]:
# Remove newline characters ('\r', '\n')
df_cleaned['movie_info'] = df['movie_info'].str.replace('\r', ' ').str.replace('\n', ' ').astype(str)

C:\Users\Main\AppData\Local\Temp\ipykernel_22868\3008832946.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cleaned['movie_info'] = df['movie_info'].str.replace('\r', ' ').str.replace('\n', ' ').astype(str)


In [28]:
df_cleaned['movie_info'].head()

0                                       Always trouble-prone, the life of teenager Percy Jackson (Logan Lerman) gets a lot more complicated when he learns he's the son of the Greek god Poseidon. At a training ground for the children of deities, Percy learns to harness his divine powers and prepare for the adventure of a lifetime: he must prevent a feud among the Olympians from erupting into a devastating war on Earth, and rescue his mother from the clutches of Hades, god of the underworld.
1       Kate (Catherine Keener) and her husband Alex (Oliver Platt) are wealthy New Yorkers who prowl estate sales and make a tidy profit reselling items they bought cheaply. They buy the apartment next door and plan to remodel just as soon as its current occupant, a cranky old woman, dies. Kate is troubled by the way she and her husband earn a living, and tries to assuage her guilt by befriending her tenant and the woman's granddaughters, but her overtures lead to unexpected consequences.
2         

In [29]:
#df_cleaned

In [30]:
# Combining relevant columns for cleaning and then LSH model training
df_cleaned['movie_combined'] = df['movie_title'] + ' ' + df['movie_info']+ ' ' + df['content_rating']+ ' ' + df['genres']+ ' ' + df['directors']+ ' ' + df['actors']

C:\Users\Main\AppData\Local\Temp\ipykernel_22868\4072557775.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cleaned['movie_combined'] = df['movie_title'] + ' ' + df['movie_info']+ ' ' + df['content_rating']+ ' ' + df['genres']+ ' ' + df['directors']+ ' ' + df['actors']


In [31]:
df_cleaned

,movie_title,movie_info,content_rating,genres,directors,actors,movie_combined
0,Percy Jackson & the Olympians: The Lightning Thief,"Always trouble-prone, the life of teenager Percy Jackson (Logan Lerman) gets a lot more complicated when he learns he's the son of the Greek god Poseidon. At a training ground for the children of deities, Percy learns to harness his divine powers and prepare for the adventure of a lifetime: he must prevent a feud among the Olympians from erupting into a devastating war on Earth, and rescue his mother from the clutches of Hades, god of the underworld.",PG,"Action & Adventure, Comedy, Drama, Science Fiction & Fantasy",Chris Columbus,"Logan Lerman, Brandon T. Jackson, Alexandra Daddario, Jake Abel","Percy Jackson & the Olympians: The Lightning Thief Always trouble-prone, the life of teenager Percy Jackson (Logan Lerman) gets a lot more complicated when he learns he's the son of the Greek god Poseidon. At a training ground for the children of deities, Percy learns to harness his divine powers and prepare for the adventure of a lifetime: he must prevent a feud among the Olympians from erupting into a devastating war on Earth, and rescue his mother from the clutches of Hades, god of the underworld. PG Action & Adventure, Comedy, Drama, Science Fiction & Fantasy Chris Columbus Logan Lerman, Brandon T. Jackson, Alexandra Daddario, Jake Abel"
1,Please Give,"Kate (Catherine Keener) and her husband Alex (Oliver Platt) are wealthy New Yorkers who prowl estate sales and make a tidy profit reselling items they bought cheaply. They buy the apartment next door and plan to remodel just as soon as its current occupant, a cranky old woman, dies. Kate is troubled by the way she and her husband earn a living, and tries to assuage her guilt by befriending her tenant and the woman's granddaughters, but her overtures lead to unexpected consequences.",R,Comedy,Nicole Holofcener,"Catherine Keener, Amanda Peet, Oliver Platt, Rebecca Hall","Please Give Kate (Catherine Keener) and her husband Alex (Oliver Platt) are wealthy New Yorkers who prowl estate sales and make a tidy profit reselling items they bought cheaply. They buy the apartment next door and plan to remodel just as soon as its current occupant, a cranky old woman, dies. Kate is troubled by the way she and her husband earn a living, and tries to assuage her guilt by befriending her tenant and the woman's granddaughters, but her overtures lead to unexpected consequences. R Comedy Nicole Holofcener Catherine Keener, Amanda Peet, Oliver Platt, Rebecca Hall"
2,10,"A successful, middle-aged Hollywood songwriter falls hopelessly in love with the woman of his dreams, and even follows the girl and her new husband to their Mexican honeymoon resort. While his behavior seems sure to land him in trouble, out of the blue fate plays into his hands.",R,"Comedy, Romance",Blake Edwards,"Dudley Moore, Bo Derek, Julie Andrews, Robert Webber","10 A successful, middle-aged Hollywood songwriter falls hopelessly in love with the woman of his dreams, and even follows the girl and her new husband to their Mexican honeymoon resort. While his behavior seems sure to land him in trouble, out of the blue fate plays into his hands. R Comedy, Romance Blake Edwards Dudley Moore, Bo Derek, Julie Andrews, Robert Webber"
3,12 Angry Men (Twelve Angry Men),"Following the closing arguments in a murder trial, the 12 members of the jury must deliberate, with a guilty verdict meaning death for the accused, an inner-city teen. As the dozen men try to reach a unanimous decision while sequestered in a room, one juror (Henry Fonda) casts considerable doubt on elements of the case. Personal issues soon rise to the surface, and conflict threatens to derail the delicate process that will decide one boy's fate.",NR,"Classics, Drama",Sidney Lumet,"Martin Balsam, John Fiedler, Lee J. Cobb, E.G. Marshall","12 Angry Men (Twelve Angry Men) Following the closing arguments in a murder trial, the 12 members of the jury mu

In [32]:
#Function to clean_text
def clean_text(text):
    lem = WordNetLemmatizer()
    stop = set(stopwords.words('english'))
    punct = string.punctuation
    text = re.sub(r'\s+', ' ', text)
    text = text.translate(str.maketrans('', '', punct)).lower()
    tokens = re.split(r'\W+', text)
    tokens = [lem.lemmatize(word) for word in tokens if word not in stop]
    return ' '.join(tokens)

In [33]:
#Use function to clean the movie_combined column
df_cleaned['movie_combined'] = df_cleaned['movie_combined'].apply(clean_text)
df_cleaned['movie_combined'].head()

C:\Users\Main\AppData\Local\Temp\ipykernel_22868\821419756.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cleaned['movie_combined'] = df_cleaned['movie_combined'].apply(clean_text)


0      percy jackson olympian lightning thief always troubleprone life teenager percy jackson logan lerman get lot complicated learns he son greek god poseidon training ground child deity percy learns harness divine power prepare adventure lifetime must prevent feud among olympian erupting devastating war earth rescue mother clutch hades god underworld pg action adventure comedy drama science fiction fantasy chris columbus logan lerman brandon jackson alexandra daddario jake abel
1                                              please give kate catherine keener husband alex oliver platt wealthy new yorkers prowl estate sale make tidy profit reselling item bought cheaply buy apartment next door plan remodel soon current occupant cranky old woman dy kate troubled way husband earn living try assuage guilt befriending tenant woman granddaughter overture lead unexpected consequence r comedy nicole holofcener catherine keener amanda peet oliver platt rebecca hall
2                             

# Shingles Size is Fixed

In [34]:
# Creating function to create shingles
def create_shingles(text, shingle_size=6):
    return [text[i:i+shingle_size] for i in range(len(text)-shingle_size+1)]

# Creating function to generate MinHash Forest (using fixed shingle size)

- Initializing number of permutations in MinHash
- MinHashing the string on all shingles in each document
- Storing the MinHash of the string
- Generating a forest of all MinHashed strings
- Indexing the forest to make it searchable

In [35]:
def generate_forest(docs, permutations):
    start_time = time.time()

    minhash = []

    for doc in docs:
        shingles = create_shingles(doc)
        m = MinHash(num_perm=permutations)
        for shingle in shingles:
            m.update(shingle.encode('utf8'))
        minhash.append(m)

    forest = MinHashLSHForest(num_perm=permutations)

    for i,m in enumerate(minhash):
        forest.add(i,m)

    forest.index()

    print('It took %s seconds to build forest.' %(time.time()-start_time))

    return forest

#Creating a  function to query MinHash Forest (using fixed shingle size)

- Preprocessing input text into fixed size shingles
- Using the same number of permutations for the MinHash as was used to build the forest
- Creating a MinHash on the input text using all shingles
- Querying the forest with MinHash and return the number of requested recommendations
- Providing the titles of each conference paper recommended

In [53]:
def predict(idx, df_cleaned, permutations, num_results, forest):
    """
    Find num_results nearest neighbors for the movie at index idx,
    excluding the movie itself from the recommendations.
    """
    text = df_cleaned.loc[idx, 'movie_combined']  # or any text field you used for creating shingles
    shingles = create_shingles(text)
    m = MinHash(num_perm=permutations)
    for shingle in shingles:
        m.update(shingle.encode('utf8'))

    # Query for num_results + 1 because the movie itself will be returned
    idx_array = np.array(forest.query(m, num_results + 1))
    if len(idx_array) == 0:
        return None


    # Remove the movie itself from the results
    idx_array = idx_array[idx_array != idx]

    result = df_cleaned.iloc[idx_array][['movie_title', 'content_rating', 'genres', 'directors', 'actors']]
    return result


# Creating Forest (Using Fixed Shingle Size)

In [37]:
# Set number of Permutations
permutations = 128

In [38]:
# Creating forest (using fixed shingle size)
forest = generate_forest(df_cleaned['movie_combined'], permutations)

It took 35.34300398826599 seconds to build forest.


# Using forest (with fixed shingle size) to make recommendations for one/more movies using 2 approaches. First is index and second is using Movie title

## Testing with dynamic approach 

First lets locate the move record based on the title 

In [39]:
def search_movies(df, search_string):
    """
    Searches for movie titles containing the given search string (case-insensitive).
    Returns a DataFrame with the matching rows and their original indexes.
    """
    mask = df['movie_title'].str.contains(search_string, case=False, na=False)
    result_df = df[mask].copy()
    return result_df[['movie_title', 'directors']]


In [40]:
search_list = search_movies(df_cleaned, "Die hard")
search_list

,movie_title,directors
2133,A Good Day To Die Hard,John Moore (VIII)
5493,Die Hard,John McTiernan
5494,Die Hard 2,Renny Harlin
5495,Die Hard: With a Vengeance,John McTiernan
9614,Live Free or Die Hard,Len Wiseman


Now that we know what movie we want to search for, lets do a search for the top 5 recommendations 

In [54]:
idx = 5493
num_recommendations = 5
input_title = df_cleaned.loc[idx]['movie_title']
input_text = df_cleaned.loc[idx]['movie_combined']
results = predict(idx, df_cleaned, permutations, num_recommendations, forest)
movie_info = df_cleaned.loc[idx, ['movie_title', 'content_rating', 'genres', 'directors', 'actors', 'movie_combined']]
print("Input Movie:\n", movie_info)
print(f'\nTop {num_recommendations} recommendations for [{input_title}]:')
results

Input Movie:
 movie_title                                                                                                                                                                                                                                                                                                                                                                                                                           Die Hard
content_rating                                                                                                                                                                                                                                                                                                                                                                                                                               R
genres                                                                                                                      

,movie_title,content_rating,genres,directors,actors
8033,"Hush...Hush, Sweet Charlotte",NR,"Classics, Horror, Mystery & Suspense",Robert Aldrich,"Bette Davis, Olivia de Havilland, Agnes Moorehead, Joseph Cotten"
7195,A Good Woman,PG,"Comedy, Drama, Romance",Mike Barker,"Helen Hunt, Scarlett Johansson, Tom Wilkinson, Stephen Campbell Moore"
5493,Die Hard,R,"Action & Adventure, Mystery & Suspense",John McTiernan,"Bruce Willis, Alan Rickman, Reginald VelJohnson, Bonnie Bedelia"
11908,Pledge,NR,"Horror, Mystery & Suspense",Daniel Robbins,"Zachery Byrd, Aaron Dalla Villa, Cameron Cowperthwaite, Jesse Pimentel"
15013,The Garden of the Finzi-Continis (Il Giardino dei Finzi-Contini),R,"Art House & International, Drama",Vittorio De Sica,"Dominique Sanda, Lino Capolicchio, Helmut Berger, Fabio Testi"
16512,Trouble Every Day (Gargoyle),NR,"Art House & International, Drama, Horror",Claire Denis,"Vincent Gallo, Tricia Vessey, Beatrice Dalle, Alex Descas"
